<a href="https://colab.research.google.com/github/listarova-aleksandra/ml_projects/blob/main/lab2_classification_astro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторная работа "Методы классификации"


##Задание 1. Аэросейсмология.
**В наборе представлены данные по звездам с указанием их эволюционных фаз.**
***
**Columns description:**

Kpmag --mag-- Kepler apparent magnitude

numax --uHz-- Frequency of maximum oscillation power

e_numax --uHz-- Uncertainty in numax

Delnu --uHz-- Mean frequency separation of acoustic modes

e_Delnu --uHz-- Uncertainty in Delnu

A --ppm-- Oscillation amplitude per radial mode

e_A --ppm-- Uncertainty in A

Width --uHz-- Power excess width

e_Width --uHz-- Uncertainty in Width

Teff --K-- Effective temperature

e_Teff --K-- Uncertainty in Teff

log(g) --[cm/s2]-- Log surface gravity from this work

e_log(g) --[cm/s2]-- Uncertainty in log(g)

[Fe/H] --[Sun]-- Metallicity

e_[Fe/H] --[Sun]-- Uncertainty in [Fe/H]

Gran --ppm2/uHz-- Granulation power at numax

e_Gran --ppm2/uHz-- Uncertainty in Gran

NoCorM --Msun-- No {Delta}{nu} correction mass solution

e_NoCorM --Msun-- Uncertainty in NoCorM

NoCorR --Rsun-- No {Delta}{nu} correction radius solution

e_NoCorR --Rsun-- Uncertainty in NoCorR

RGBcorM --Msun-- RGB {Delta}{nu} corrected mass solution

e_RGBcorM --Msun-- Uncertainty in RGBcorM

RGBcorR --Rsun-- GB {Delta}{nu} corrected radius solution

e_RGBcorR --Rsun-- Uncertainty in RGBcorR

ClcorM --Msun-- Clump {Delta}{nu} corrected mass solution

e_ClcorM --Msun-- Uncertainty in ClcorM

ClcorR --Rsun-- Clump {Delta}{nu} corrected radius solution

e_ClcorR --Rsun-- Uncertainty in ClcorR

Phase --- Evolutionary phase

Phase as follows:
2 = HeB phase;
1 = RGB phase;
0 = unclassified phase
RGB (Red-Giant-Branch)
HeB (Helium Burning)

***

###Первичное знакомство с датасетом и быстрый анализ данных.

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [26]:
df = pd.read_csv('Data_classified_phase.csv')

In [27]:
pd.set_option('display.max_columns', None)

In [28]:
df.head()

,Kpmag,numax,e_numax,Delnu,e_Delnu,A,e_A,Width,e_Width,Teff,e_Teff,log(g),e_log(g),[Fe/H],e_[Fe/H],NoCorM,e_NoCorM,NoCorR,e_NoCorR,RGBcorM,e_RGBcorM,RGBcorR,e_RGBcorR,ClcorM,e_ClcorM,ClcorR,e_ClcorR,Phase
0,9.20,29.99,0.60,3.399,0.011,104.9,4.6,12.3,1.5,4751,139,2.383,0.012,-0.08,0.30,1.70,0.14,13.91,0.41,1.55,0.13,13.26,0.39,1.72,0.14,13.97,0.41,1
1,13.23,29.48,0.48,3.962,0.116,149.7,8.3,12.0,3.9,5188,183,2.394,0.011,-0.21,0.30,1.00,0.14,10.51,0.68,1.04,0.15,10.74,0.70,1.00,0.14,10.52,0.68,2
2,12.58,41.39,0.54,4.311,0.013,86.1,4.6,15.3,1.6,4728,80,2.522,0.008,-0.15,0.15,1.72,0.10,11.90,0.26,1.54,0.09,11.29,0.24,1.72,0.10,11.92,0.26,1
3,12.14,41.17,0.90,4.414,0.061,63.8,2.9,24.8,2.7,5072,166,2.534,0.013,-0.12,0.30,1.71,0.18,11.69,0.48,1.67,0.17,11.55,0.48,1.75,0.18,11.83,0.49,2
4,11.74,36.91,0.71,3.991,0.064,116.1,9.6,14.0,2.4,4718,148,2.471,0.012,-0.02,0.30,1.65,0.17,12.37,0.53,1.50,0.16,11.80,0.50,1.67,0.17,12.43,0.53,2


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15388 entries, 0 to 15387
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Kpmag      15388 non-null  float64
 1   numax      15388 non-null  float64
 2   e_numax    15388 non-null  float64
 3   Delnu      15388 non-null  float64
 4   e_Delnu    15388 non-null  float64
 5   A          15388 non-null  object 
 6   e_A        15388 non-null  object 
 7   Width      15388 non-null  object 
 8   e_Width    15388 non-null  object 
 9   Teff       15388 non-null  int64  
 10  e_Teff     15388 non-null  int64  
 11  log(g)     15388 non-null  float64
 12  e_log(g)   15388 non-null  float64
 13  [Fe/H]     15388 non-null  float64
 14  e_[Fe/H]   15388 non-null  float64
 15  NoCorM     15388 non-null  float64
 16  e_NoCorM   15388 non-null  float64
 17  NoCorR     15388 non-null  float64
 18  e_NoCorR   15388 non-null  float64
 19  RGBcorM    15388 non-null  float64
 20  e_RGBc

In [30]:
df.describe()

,Kpmag,numax,e_numax,Delnu,e_Delnu,Teff,e_Teff,log(g),e_log(g),[Fe/H],e_[Fe/H],NoCorM,e_NoCorM,NoCorR,e_NoCorR,RGBcorM,e_RGBcorM,RGBcorR,e_RGBcorR,ClcorM,e_ClcorM,ClcorR,e_ClcorR,Phase
count,15388.000000,15388.000000,15388.000000,15388.00000,15388.000000,15388.00000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000,15388.00000,15388.000000,15388.000000,15388.000000,15388.000000,15388.000000
mean,12.632402,64.673195,0.903276,6.16765,0.047327,4903.08994,122.429036,2.617776,0.011179,-0.129281,0.236186,1.403965,0.132816,9.976145,0.407304,1.344886,0.126670,9.720831,0.39403,1.422839,0.134646,10.024309,0.409406,1.500585
std,1.211778,50.958690,0.891080,3.56072,0.069763,212.91843,36.123948,0.295648,0.005376,0.311975,0.075506,0.390115,0.111342,3.231068,0.452677,0.409900,0.106109,3.053634,0.42836,0.417293,0.114116,3.216817,0.453516,0.500016
min,6.240000,3.970000,0.040000,0.74700,0.008000,3225.00000,50.000000,1.476000,0.006000,-2.500000,0.100000,0.410000,0.040000,3.540000,0.060000,0.340000,0.030000,3.240000,0.06000,0.430000,0.040000,3.470000,0.060000,1.000000
25%,11.900000,31.680000,0.560000,3.97675,0.017000,4769.00000,80.000000,2.413000,0.008000,-0.290000,0.150000,1.170000,0.080000,8.130000,0.180000,1.100000,0.080000,7.880000,0.17000,1.170000,0.090000,8.160000,0.180000,1.000000
50%,12.850000,42.260000,0.720000,4.54050,0.028000,4907.00000,138.000000,2.538000,0.010000,-0.100000,0.300000,1.340000,0.110000,10.480000,0.310000,1.270000,0.100000,10.220000,0.30000,1.350000,0.110000,10.520000,0.310000,2.000000
75%,13.510000,79.522500,1.000000,7.27000,0.050000,5047.00000,155.000000,2.816000,0.012000,0.080000,0.300000,1.550000,0.140000,11.390000,0.460000,1.480000,0.130000,11.160000,0.45000,1.580000,0.140000,11.450000,0.460000,2.000000
max,16.850000,273.160000,70.270000,19.29400,1.338000,6356.00000,682.000000,3.357000,0.134000,1.000000,1.600000,4.560000,2.820000,48.360000,9.800000,5.150000,2.800000,48.610000,9.00000,5.240000,2.950000,51.190000,9.800000,2.000000


In [31]:
df['Phase'].value_counts()

,count
Phase,
2,7703
1,7685


**Быстрые выводы:**

**1. Нет пропусков в значениях, что не может не радовать.**

**2. В четрых столбцах тип данных object, хотя там числовые значения. Надо проверить эти данные на пропуски и поменять тип данных на float64.**

**3. Наблюдаю хорошее распределение классов в target - столбце , почти равное: 7703/7685.**

###Приведение данных в столбцах  'A', 'e_A', 'Width', 'e_Width' к нужному формату.

In [32]:
df.columns

Index(['Kpmag', 'numax', 'e_numax', 'Delnu', 'e_Delnu', 'A', 'e_A', 'Width',
       'e_Width', 'Teff', 'e_Teff', 'log(g)', 'e_log(g)', '[Fe/H]', 'e_[Fe/H]',
       'NoCorM', 'e_NoCorM', 'NoCorR', 'e_NoCorR', 'RGBcorM', 'e_RGBcorM',
       'RGBcorR', 'e_RGBcorR', 'ClcorM', 'e_ClcorM', 'ClcorR', 'e_ClcorR',
       'Phase'],
      dtype='object')

In [33]:
df['A'].value_counts()

,count
A,
,537
87.5,26
100.7,24
94.0,23
43.5,23
...,...
228.5,1
271.4,1
381.0,1


**Видно, что есть строковые пробелы в данных, из-за которых на df.info() столбец принимает тип object.**

In [34]:
object_columns = [ 'A', 'e_A', 'Width', 'e_Width']
for column in object_columns:
    df[column] = pd.to_numeric(df[column], 'coerce')
    df[column] = df[column].astype(float)

df[object_columns].isnull().sum()

,0
A,537
e_A,537
Width,537
e_Width,537


In [35]:
df.tail()

,Kpmag,numax,e_numax,Delnu,e_Delnu,A,e_A,Width,e_Width,Teff,e_Teff,log(g),e_log(g),[Fe/H],e_[Fe/H],NoCorM,e_NoCorM,NoCorR,e_NoCorR,RGBcorM,e_RGBcorM,RGBcorR,e_RGBcorR,ClcorM,e_ClcorM,ClcorR,e_ClcorR,Phase
15383,12.78,37.81,0.42,4.335,0.103,108.0,4.3,14.4,1.0,4906,80,2.490,0.007,-0.09,0.15,1.35,0.15,10.95,0.56,1.29,0.14,10.68,0.54,1.37,0.15,11.03,0.57,2
15384,13.69,28.74,0.78,4.054,0.057,147.8,6.7,14.6,2.0,5077,154,2.379,0.014,-0.53,0.30,0.82,0.09,9.68,0.43,0.83,0.09,9.73,0.43,0.84,0.10,9.80,0.43,2
15385,12.60,92.86,0.76,8.187,0.015,61.8,3.0,31.7,3.3,4846,142,2.878,0.008,0.07,0.30,1.55,0.10,7.50,0.17,1.46,0.09,7.29,0.16,1.57,0.10,7.55,0.17,1
15386,11.84,52.02,0.45,5.354,0.010,86.6,3.9,16.2,0.9,4855,144,2.627,0.009,0.12,0.30,1.49,0.09,9.83,0.22,1.42,0.09,9.59,0.22,1.51,0.10,9.90,0.22,1
15387,13.11,30.03,0.66,3.891,0.049,148.6,5.8,12.3,2.2,4834,143,2.387,0.012,0.01,0.30,1.02,0.10,10.72,0.42,0.99,0.10,10.57,0.41,1.03,0.10,10.74,0.42,2


In [36]:
df = df.dropna(subset=['Phase'])

###Метрики

In [37]:
def metrics(y_true, y_predictable):
    accuracy = accuracy_score(y_true, y_predictable)
    precision = precision_score(y_true, y_predictable)
    recall = recall_score(y_true, y_predictable)
    f1 = f1_score(y_true, y_predictable)

    print('Оценки модели')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'f1: {f1}')

###Разделение данных

**Пока попробую тетсово без отбора признаков. Но если модель будет показывать плохой результат, то допишу это.**

In [38]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Phase'], axis = 1), df['Phase'], test_size=0.2, random_state=22)

**Теперь перехожу к созданию моделей. Буду использовать три модели классификации:**

**1. Random forest. Здесь немаленькое количество данных. Также у эта модель довольно стабильна и обладает высокой точностью.**

**2. SVM. У этого метода довольно так же высокая точность.**

**3. LDA. Она довольно хорошо работает с большим количеством данных.**

**Также сюда добавлю GridSearch для подбора парметров для каждой модели.**

In [39]:
models = {
    "Random Forest" : {
        "model" : RandomForestClassifier(random_state=22),
        "params" : {
            'classifier__n_estimators' : [50, 100, 150],
            'classifier__max_features' : ['sqrt', 'log2', None]
        }
    },
    "SVM" : {
        "model" : SVC(random_state=22),
        "params" : {
            'classifier__gamma' : ['scale', 'auto'],
            'classifier__kernel' : ['linear', 'rbf', 'poly']
        }
    },
    "LDA" : {
        "model" : LinearDiscriminantAnalysis(),
        "params" : {
            'classifier__solver' : ['svd', 'lsqr'],
            'classifier__shrinkage' : [None, 'auto']
        }
    }
}

###Pipeline

**Теперь с помощью pipeline объединяю этапы обработки данных:**

**1. Заполняю пропуски средними значениями.**

**2. Нормализую данные.**

**3. Циклом прохожу по моделям, выполняя GridSearch с ранее заданным параметрами.**

**4. Делаю предсказание на тесте и смотрю метрики.**

In [40]:
for name, config in models.items():
    print(name)
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('classifier', config['model'])
    ])

    gridsearch =GridSearchCV(pipeline, config['params'], cv = 5, scoring='f1')
    gridsearch.fit(x_train, y_train)

    print(f'Лучшая модель: {gridsearch.best_params_}')
    print(f'Лучший f1 на кросс-валидации: {gridsearch.best_score_}')

    y_pred = gridsearch.predict(x_test)
    metrics(y_test, y_pred)

Random Forest
Лучшая модель: {'classifier__max_features': 'log2', 'classifier__n_estimators': 150}
Лучший f1 на кросс-валидации: 0.959723979824801
Оценки модели
Accuracy: 0.9629629629629629
Precision: 0.9647749510763209
Recall: 0.9610136452241715
f1: 0.962890625
SVM
Лучшая модель: {'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Лучший f1 на кросс-валидации: 0.9612379193685815
Оценки модели
Accuracy: 0.9619883040935673
Precision: 0.9640992167101827
Recall: 0.9597141000649773
f1: 0.9619016606968415
LDA
Лучшая модель: {'classifier__shrinkage': None, 'classifier__solver': 'svd'}
Лучший f1 на кросс-валидации: 0.9161904412930009
Оценки модели
Accuracy: 0.9165042235217674
Precision: 0.9233817701453104
Recall: 0.9083820662768031
f1: 0.9158205044218801


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last

***

**Random Forest**

Лучшая модель: {'classifier__max_features': 'log2', 'classifier__n_estimators': 150}

Лучший f1 на кросс-валидации: 0.959723979824801

**Оценки модели**

Accuracy: 0.9629629629629629

Precision: 0.9647749510763209

Recall: 0.9610136452241715

f1: 0.962890625

***
**SVM**

Лучшая модель: {'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}

Лучший f1 на кросс-валидации: 0.9612379193685815

**Оценки модели**

Accuracy: 0.9619883040935673

Precision: 0.9640992167101827

Recall: 0.9597141000649773

f1: 0.9619016606968415
***
**LDA**

Лучшая модель: {'classifier__shrinkage': None, 'classifier__solver': 'svd'}

Лучший f1 на кросс-валидации: 0.9161904412930009

**Оценки модели**

Accuracy: 0.9165042235217674

Precision: 0.9233817701453104

Recall: 0.9083820662768031

f1: 0.9158205044218801
***

###Чуть больше о метриках, которые использую для оценки моделей

**Использую кросс-валидацию на пяти фолдах на train, чтобы получить макисмально честную оценку моделей. Оцениваю модели на кросс-валидации по метрике f1_score, так как это среднее precision и recall, что дает сбалансированную оценку.**

**Что показывают выбранные метрики в контексте задачи?**

**Accuracy показывает насколько хорошо модель определяет эволюционную фазу звезды.**

**Precision: из скольких звезд, которые я отнесла к определенному классу, действительно этого класса.**

**Recall: сколько из всех звезд каждого класса я распознала.**

**Мне кажется, что в условиях этой задачи нельзя отдавать предпочтение или precision, или recall. Поэтому метрика f1_score, возможн, справляется лучше всего.**

###Выводы о полученных моделях.

**1. Еще до тестирования моделей я думала, что у Random Forest метрики будут сильно лучше, чем у двух других выбранных моделей. Но у SVM также очень хорошие метрики. Возможно, на это повлияло то, что в качестве ядра на GridSearch было выбрано RBF - ядро с самым сложным распрделением.**

**2. Может быть, метрики получились хорошими из-за того, что надо разделить всего лишь на два класса.**